<a href="https://colab.research.google.com/github/MELRIAN1910/MachineLearning/blob/main/Practice%20Projects/18_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [34]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data Collection and Pre-Processing

In [35]:
#Loading data
movies_data = pd.read_csv('/content/drive/MyDrive/Datasets/Movies.csv')

In [36]:
#top 5 data
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [37]:
#row and column count
movies_data.shape

(4803, 24)

In [38]:
#Filtering relevant features
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [39]:
#replacing Null values with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [40]:
#combining selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [41]:
# Text Data --> feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

# Cosine Similarity

In [42]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [43]:
print(similarity.shape)

(4803, 4803)


# Getting Movie name from user

In [44]:
movie = input('Enter movie name: ')

Enter movie name: Mr. & Mrs. Smith


In [45]:
#creating list with all movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [46]:
#finding the close match
find_close_match = difflib.get_close_matches(movie, list_of_all_titles)
print(find_close_match)

['Mr. & Mrs. Smith']


In [47]:
close_match = find_close_match[0]
print(close_match)

Mr. & Mrs. Smith


In [48]:
#finding movie index
index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_movie)

248


In [49]:
#Similar movie list
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, np.float64(0.006465085733297934)), (1, np.float64(0.005642740161609565)), (2, np.float64(0.052644060531429664)), (3, np.float64(0.10122601149966234)), (4, np.float64(0.006295361147583542)), (5, np.float64(0.03762213261934355)), (6, np.float64(0.0)), (7, np.float64(0.006525665488667192)), (8, np.float64(0.0)), (9, np.float64(0.0063755776883889625)), (10, np.float64(0.007271733065406198)), (11, np.float64(0.09103265446303221)), (12, np.float64(0.0066011826522979706)), (13, np.float64(0.005959950902115528)), (14, np.float64(0.006422958327220803)), (15, np.float64(0.0)), (16, np.float64(0.006187718043617793)), (17, np.float64(0.0063147975458513715)), (18, np.float64(0.010490565183420127)), (19, np.float64(0.005606515646994091)), (20, np.float64(0.006438252836650718)), (21, np.float64(0.026953609773272345)), (22, np.float64(0.0)), (23, np.float64(0.0)), (24, np.float64(0.05055006397366274)), (25, np.float64(0.009415346787437204)), (26, np.float64(0.006735054297357538)), (27, np.float64

In [50]:
len(similarity_score)

4803

In [51]:
#sorting movies(similarity score)
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(248, np.float64(1.0)), (2257, np.float64(0.2330446964845061)), (694, np.float64(0.21733595447957937)), (512, np.float64(0.18318383438145955)), (4671, np.float64(0.18275170165716498)), (339, np.float64(0.1656984676108372)), (196, np.float64(0.1592586734918223)), (2398, np.float64(0.15294422651115952)), (219, np.float64(0.14474596423900352)), (593, np.float64(0.140954539846296)), (607, np.float64(0.1396469880030222)), (1635, np.float64(0.1298271431060568)), (594, np.float64(0.12902285044604736)), (447, np.float64(0.12786410342871252)), (1009, np.float64(0.1267790344796836)), (1258, np.float64(0.12547691257403906)), (1688, np.float64(0.12543646680372822)), (991, np.float64(0.12252059434619904)), (941, np.float64(0.12139669118441403)), (525, np.float64(0.12121735216036063)), (1720, np.float64(0.1204672091327576)), (2381, np.float64(0.11964602017877776)), (295, np.float64(0.1179078029984475)), (3623, np.float64(0.1175344674138967)), (695, np.float64(0.11379344297625746)), (3169, np.float6

In [52]:
#similar movies
print('Movies suggested for you: \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you: 

1 . Mr. & Mrs. Smith
2 . By the Sea
3 . The Bourne Identity
4 . Wanted
5 . Swingers
6 . The Incredibles
7 . Megamind
8 . Hitman
9 . Charlie's Angels: Full Throttle
10 . The Dilemma
11 . Sky Captain and the World of Tomorrow
12 . The Replacement Killers
13 . Bad Company
14 . Eagle Eye
15 . The Bone Collector
16 . Life or Something Like It
17 . Confessions of a Dangerous Mind
18 . Fair Game
19 . 13 Hours: The Secret Soldiers of Benghazi
20 . Shark Tale
21 . Kick-Ass
22 . Salton Sea
23 . The Tourist
24 . Made
25 . GoldenEye
26 . The Purge: Anarchy
27 . Jumper
28 . The Big Short
29 . Girl 6


# Movie Recommendation Sytem

In [53]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : hangover
Movies suggested for you : 

1 . The Hangover
2 . The Hangover Part II
3 . Failure to Launch
4 . Cedar Rapids
5 . Due Date
6 . Last Vegas
7 . Youth in Revolt
8 . Fun Size
9 . Weekend
10 . Say It Isn't So
11 . Texas Rangers
12 . The Big Hit
13 . Trust the Man
14 . Go
15 . Paul Blart: Mall Cop 2
16 . Dinner for Schmucks
17 . The Muppets
18 . We're the Millers
19 . Forgetting Sarah Marshall
20 . Meet Dave
21 . The Campaign
22 . Austin Powers: The Spy Who Shagged Me
23 . Tim and Eric's Billion Dollar Movie
24 . Aloha
25 . Vampires Suck
26 . American Wedding
27 . Gigli
28 . Silver Linings Playbook
29 . Birdman
